In [59]:
from pathlib import Path
from typing import List, Optional, Union

import pandas as pd
import numpy as np

from gluonts.dataset.arrow import ArrowWriter

In [8]:
df = pd.read_csv("/Users/andy/Desktop/interactive_broker/data/etf_S_full_1min_adjsplitdiv_1py2dog/SPY_full_1min_adjsplitdiv.txt",
                 header=None,
                 names=['dateTime', 'open', 'high', 'low', 'close', 'volume'])
df

,dateTime,open,high,low,close,volume
0,2000-01-03 09:31:00,101.0092,101.0092,101.0092,101.0092,253029.0
1,2000-01-03 09:32:00,101.0092,101.0092,100.8815,101.0092,219126.0
2,2000-01-03 09:33:00,100.9666,101.0092,100.7111,100.7111,63257.0
3,2000-01-03 09:34:00,101.0092,101.0092,100.6260,100.6260,29647.0
4,2000-01-03 09:35:00,100.7324,101.0092,100.6473,100.7111,18346.0
...,...,...,...,...,...,...
4159117,2024-06-10 19:48:00,535.0800,535.1100,535.0800,535.1100,875.0
4159118,2024-06-10 19:54:00,535.1000,535.1000,535.1000,535.1000,153.0
4159119,2024-06-10 19:56:00,535.1900,535.1900,535.1900,535.1900,191.0
4159120,2024-06-10 19:58:00,535.1400,535.1400,535.1400,535.1400,100.0


In [9]:
# clean up datetime format
df["dateTime"] = pd.to_datetime(df["dateTime"])
df["dateTime"]

0         2000-01-03 09:31:00
1         2000-01-03 09:32:00
2         2000-01-03 09:33:00
3         2000-01-03 09:34:00
4         2000-01-03 09:35:00
                  ...        
4159117   2024-06-10 19:48:00
4159118   2024-06-10 19:54:00
4159119   2024-06-10 19:56:00
4159120   2024-06-10 19:58:00
4159121   2024-06-10 19:59:00
Name: dateTime, Length: 4159122, dtype: datetime64[ns]

In [25]:
# create field
df["date"] = df["dateTime"].dt.date

# Define normal trading hour range
start_time = pd.to_datetime('09:30').time()
end_time = pd.to_datetime('16:00').time()

def filter_time(row):
    time = row.time()
    return start_time <= time <= end_time

# Apply the filter
df = df[df['dateTime'].apply(filter_time)]

In [26]:
df.isna().any()

dateTime    False
open        False
high        False
low         False
close       False
volume      False
date        False
dtype: bool

In [29]:
# with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
#     print(df.head(1000))

In [34]:
import datetime

filter_date = datetime.date(2000, 1, 4)
df.loc[df["date"] == filter_date, ]

,dateTime,open,high,low,close,volume,date
394,2000-01-04 09:33:00,97.7941,97.7941,97.7941,97.7941,282823.0,2000-01-04
395,2000-01-04 09:34:00,97.7941,97.7941,97.7941,97.7941,105233.0,2000-01-04
396,2000-01-04 09:35:00,97.7941,97.9006,97.7728,97.9006,48727.0,2000-01-04
397,2000-01-04 09:36:00,97.8154,97.9645,97.8154,97.9645,19520.0,2000-01-04
398,2000-01-04 09:37:00,97.9432,97.9432,97.7303,97.7728,43444.0,2000-01-04
...,...,...,...,...,...,...,...
774,2000-01-04 15:56:00,95.2604,95.3668,95.2391,95.3668,35665.0,2000-01-04
775,2000-01-04 15:57:00,95.2604,95.3881,95.2604,95.3243,50782.0,2000-01-04
776,2000-01-04 15:58:00,95.3243,95.4307,95.3030,95.3881,88795.0,2000-01-04
777,2000-01-04 15:59:00,95.3881,95.4307,95.3455,95.4307,29794.0,2000-01-04


In [43]:
MAX_SERIES_LENGTH = 1024

time_series_df = df.groupby("date").apply(lambda x: x["open"].to_numpy(), include_groups=False).reset_index().rename(columns={0: "time_series"})
time_series_df["time_series"] = time_series_df.apply(lambda x: x["time_series"][:MAX_SERIES_LENGTH], axis=1)

In [48]:
time_series_df.apply(lambda x: len(x["time_series"]), axis=1).describe()

count    6148.000000
mean      388.715192
std        12.604238
min       187.000000
25%       391.000000
50%       391.000000
75%       391.000000
max       391.000000
dtype: float64

In [51]:
# Find the maximum length of arrays in the series
max_len = max(time_series_df["time_series"].apply(len))
time_series_padded = time_series_df["time_series"].apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=np.nan))
time_series_padded

0       [101.0092, 101.0092, 100.9666, 101.0092, 100.7...
1       [97.7941, 97.7941, 97.7941, 97.8154, 97.9432, ...
2       [95.3455, 95.3455, 95.3881, 95.3455, 95.5585, ...
3       [95.1326, 95.2178, 95.3881, 95.3881, 95.5159, ...
4       [95.6011, 95.6011, 95.8566, 95.8566, 95.814, 9...
                              ...                        
6143    [526.46, 526.5, 526.355, 526.42, 526.1, 525.85...
6144    [530.77, 530.28, 530.39, 530.32, 530.22, 530.2...
6145    [534.98, 534.88, 534.85, 534.91, 534.92, 534.9...
6146    [533.66, 532.93, 533.09, 532.96, 532.86, 532.5...
6147    [533.18, 533.1, 532.9, 533.005, 532.88, 533.03...
Name: time_series, Length: 6148, dtype: object

In [56]:
matrix = np.vstack(time_series_padded.to_numpy())
matrix.shape

(6148, 391)

In [60]:
def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    start_times: Optional[Union[List[np.datetime64], np.ndarray]] = None,
    compression: str = "lz4",
):
    if start_times is None:
        # Set an arbitrary start time
        start_times = [np.datetime64("2000-01-01 00:00", "s")] * len(time_series)

    assert len(time_series) == len(start_times)

    dataset = [
        {"start": start, "target": ts} for ts, start in zip(time_series, start_times)
    ]
    ArrowWriter(compression=compression).write_to_file(
        dataset,
        path=path,
    )


if __name__ == "__main__":
    # Generate 20 random time series of length 1024
    time_series = [np.random.randn(1024) for i in range(20)]

    # Convert to GluonTS arrow format
    convert_to_arrow("../data/spy_1min_bar.arrow", time_series=matrix)